In [2]:
# Data manipulation
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100

# Modeling
# from matrix_factorization import BaselineModel, KernelMF, train_update_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# Other
import os
import random
import sys

from letterboxdpy import user
from letterboxdpy import movie

import sqlite3
import pandas as pd

from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

import sklearn

# Get the data (database shit - oud)

In [2]:
%%time

user1 = "sverlaan"
user1_inst = user.User(user1)
wl1 = {movie['slug'] for movie in user1_inst.get_watchlist()['data'].values()}
get_movies1 = user1_inst.get_films()
allmovies1 = [movie for movie in get_movies1['movies'].keys()]

user2 = "liannehr"
user2_inst = user.User(user2)
wl2 = {movie['slug'] for movie in user2_inst.get_watchlist()['data'].values()}
get_movies2 = user2_inst.get_films()
allmovies2 = [movie for movie in get_movies2['movies'].keys()]

allmovies = list(wl2.union(wl1).union(allmovies2).union(allmovies1))

CPU times: user 1.75 s, sys: 70.2 ms, total: 1.82 s
Wall time: 15.7 s


In [143]:
len(allmovies)

1097

In [3]:
# Path to your SQLite database file
db_path = "bootstrap_project/ratingsdb/ratings.db"  # Change this to your actual file path
# List of movie IDs you want to filter
movie_ids = allmovies

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [4]:
%%time
# Create a dynamic SQL query using placeholders
placeholders = ", ".join(["?"] * len(movie_ids))  # Creates "?, ?, ?" for each movie_id
query = f"SELECT * FROM ratings WHERE movie_id IN ({placeholders})"

# Execute query with movie_ids as parameters
cursor.execute(query, movie_ids)

# Fetch all matching rows
results = cursor.fetchall()

# Close the connection when done
conn.close()

CPU times: user 5.12 s, sys: 270 ms, total: 5.39 s
Wall time: 5.55 s


In [42]:
%%time
# Define column names
columns = ["userId", "movieId", "rating"]

# Create DataFrame
df = pd.DataFrame(results, columns=columns)

# Print DataFrame
df.head()

CPU times: user 309 ms, sys: 20.3 ms, total: 329 ms
Wall time: 328 ms


,userId,movieId,rating
0,kurstboy,spider-man-2,4.5
1,kurstboy,poison-2023,4.5
2,kurstboy,the-rat-catcher-2023,3.5
3,kurstboy,the-swan-2023,4.0
4,kurstboy,the-boy-and-the-heron,4.5


In [10]:
unique_counts = df.nunique()
print(unique_counts)


user_id    11033
item_id     1055
rating        10
dtype: int64


# Get the data (database shit - oud)

# Setup data

In [19]:
df = pd.read_csv("bootstrap_project/ratingsdb/ratings-2.csv", dtype={
    'user_name': 'string',
    'film_id': 'string',
    'rating': 'float64'
})

df = df.rename(columns={'user_name': 'userId', 'film_id': 'movieId'})
df

,userId,movieId,rating
0,kurstboy,spider-man-2,4.5
1,kurstboy,tetsuo-the-iron-man,4.0
2,kurstboy,bram-stokers-dracula,4.5
3,kurstboy,poison-2023,4.5
4,kurstboy,the-rat-catcher-2023,3.5
...,...,...,...
18175540,beef,parasite-2019,4.0
18175541,beef,the-paper,2.5
18175542,beef,the-farewell-2019,3.5
18175543,beef,ready-or-not-2019,4.0


In [24]:
df = df.rename(columns={'user_name': 'userId', 'film_id': 'movieId'})

popular_movies = df[df.groupby("movieId")["movieId"].transform("count") >= 1000]
slugs = sorted(list(popular_movies["movieId"].unique())[:1000])
slugs

['10-things-i-hate-about-you',
 '12-angry-men',
 '127-hours',
 '13th',
 '1917',
 '20th-century-women',
 '3-women',
 '500-days-of-summer',
 '80-for-brady',
 'a-beautiful-day-in-the-neighborhood',
 'a-charlie-brown-christmas',
 'a-clockwork-orange',
 'a-goofy-movie',
 'a-grand-day-out',
 'a-hard-days-night',
 'a-hidden-life-2019',
 'a-league-of-their-own',
 'a-man-called-otto',
 'a-man-escaped',
 'a-new-leaf',
 'a-nightmare-on-elm-street',
 'a-quiet-place-2018',
 'a-simple-favor',
 'a-star-is-born-2018',
 'a-woman-is-a-woman',
 'a-woman-under-the-influence',
 'ace-in-the-hole',
 'ad-astra-2019',
 'adaptation',
 'after-hours',
 'after-yang',
 'aftersun',
 'air-2023',
 'airplane',
 'akira',
 'aladdin',
 'alice-doesnt-live-here-anymore',
 'alien',
 'all-about-my-mother',
 'all-quiet-on-the-western-front-2022',
 'all-that-jazz',
 'all-the-beauty-and-the-bloodshed',
 'allegiant',
 'alvin-and-the-chipmunks',
 'ambulance-2022',
 'amelie',
 'amour',
 'amsterdam-2022',
 'an-american-pickle',
 'an

In [44]:
df[df["movieId"] == "pickpocket"]

,userId,movieId,rating
533,kurstboy,pickpocket,4.0
2162,silentdawn,pickpocket,4.0
42009,davidlsims,pickpocket,5.0
62948,nycsubwayrat,pickpocket,4.0
69363,darrencb,pickpocket,4.5
...,...,...,...
18097221,deividrps,pickpocket,4.5
18103116,tiago_costa,pickpocket,1.5
18109841,sasori701gou,pickpocket,3.0
18135453,introvertedolaf,pickpocket,5.0


In [21]:
df = df[df.groupby("movieId")["movieId"].transform("count") >= 1000]   # only movies that more than 1000 users have seen
df = df[df.groupby("userId")["userId"].transform("count") >= 10]       # only users with more than 10 watched movies
print("Num users:", len(df["userId"].unique()))
print("Num movies:", len(df["movieId"].unique()))
df

Num users: 10989
Num movies: 4111


,userId,movieId,rating
0,kurstboy,spider-man-2,4.5
1,kurstboy,tetsuo-the-iron-man,4.0
2,kurstboy,bram-stokers-dracula,4.5
3,kurstboy,poison-2023,4.5
4,kurstboy,the-rat-catcher-2023,3.5
...,...,...,...
18175532,beef,hustlers-2019,3.0
18175538,beef,a-matter-of-life-and-death,4.0
18175540,beef,parasite-2019,4.0
18175542,beef,the-farewell-2019,3.5


In [173]:
random_users = random_users = df['userId'].drop_duplicates().sample(n=5000, replace=False)
df = df[df['userId'].isin(random_users)]

In [46]:
%%time
username1 = "liannehr"
user_inst1 = user.User(username1)
watchlist1 = {movie['slug'] for movie in user_inst1.get_watchlist()['data'].values()}
ratings1 = user_inst1.get_films()

username2 = "sverlaan"
user_inst2 = user.User(username2)
watchlist2 = {movie['slug'] for movie in user_inst2.get_watchlist()['data'].values()}
ratings2 = user_inst2.get_films()

CPU times: user 1.55 s, sys: 111 ms, total: 1.66 s
Wall time: 14.6 s


In [56]:
user_rows1 = [(username1, slug, info['rating'] / 2.0) for slug, info in ratings1['movies'].items() if info['rating'] is not None]
user_rows2 = [(username2, slug, info['rating'] / 2.0) for slug, info in ratings2['movies'].items() if info['rating'] is not None]
combined_rows = user_rows1+user_rows2

In [83]:
# Add high star ratings multiple times?
high_star_rows = [(u, m, r) for (u, m, r) in user_rows1 if r >= 4.5] * 3
combined_rows = user_rows1+high_star_rows

In [57]:
new_df = pd.DataFrame(combined_rows, columns=df.columns)
df = pd.concat([df, new_df], ignore_index=True)
df.tail()

,userId,movieId,rating
9819798,sverlaan,the-wizard-of-oz-1939,3.0
9819799,sverlaan,m,4.0
9819800,sverlaan,the-passion-of-joan-of-arc,4.0
9819801,sverlaan,metropolis,4.0
9819802,sverlaan,sherlock-jr,4.0


In [58]:
# Encode user_id and item_id
df["userId"], user_mapping = pd.factorize(df["userId"])
df["movieId"], item_mapping = pd.factorize(df["movieId"])

# Function to get user ID from username
def get_user_id(username):
    try:
        return user_mapping.tolist().index(username)
    except:
        return None

def get_user_name(id):
    try:
        return user_mapping[id]
    except:
        return None

def get_movie_id(slug):
    try:
        return item_mapping.tolist().index(slug)
    except:
        return None
        
def get_movie_name(id):
    try:
        return item_mapping[id]
    except:
        return None

In [42]:
print(get_user_id("liannehr"))
print(get_user_name(11033))

print(get_movie_id("metropolis"))
print(get_movie_name(537))

11033
liannehr
537
metropolis


In [59]:
# Get minimum and maximum rating from the dataset
min_rating = df.rating.min()
max_rating = df.rating.max()

# Find best params

In [8]:
%%time
 
svd = SVD(n_epochs=30, n_factors=100)
results = cross_validate(svd, data, measures=['mse', 'rmse'], cv=3, verbose=True)

Evaluating MSE, RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MSE (testset)     0.4396  0.4402  0.4407  0.4402  0.0004  
RMSE (testset)    0.6630  0.6635  0.6638  0.6635  0.0003  
Fit time          17.80   16.53   16.47   16.93   0.61    
Test time         4.26    3.60    3.77    3.88    0.28    
CPU times: user 1min 9s, sys: 606 ms, total: 1min 9s
Wall time: 1min 10s


In [59]:
%%time

param_grid = {
  'n_factors': [100, 150],
  'n_epochs': [20, 35]
}
 
gs = GridSearchCV(SVD, param_grid, measures=['mse', 'rmse'], cv=5)
gs.fit(data)
 
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.best_score['mse'])
print(gs.best_params['mse'])

0.6556149129434622
{'n_factors': 100, 'n_epochs': 35}
0.42983134180981153
{'n_factors': 100, 'n_epochs': 35}
CPU times: user 9min 18s, sys: 1.92 s, total: 9min 20s
Wall time: 9min 21s


# Train

In [60]:
%%time

reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.1)
 
svd = SVD(n_factors=100, n_epochs=30, verbose=True)
svd.fit(trainset)

predictions = svd.test(testset)
rmse = accuracy.rmse(predictions)
print(f"Test RMSE: {rmse}")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
RMSE: 0.6568
Test RMSE: 0.6567660455124157
CPU times: user 1min 41s, sys: 6.64 s, total: 1min 48s
Wall time: 1min 51s


# Recommend

In [10]:
watchlist1_ids = [get_movie_id(slug) for slug in watchlist1 if get_movie_id(slug) is not None]
watchlist2_ids = [get_movie_id(slug) for slug in watchlist2 if get_movie_id(slug) is not None]
common_watchlist = list(set(watchlist1_ids).intersection(set(watchlist2_ids)))
union_watchlist = list(set(watchlist1_ids).union(set(watchlist2_ids)))

In [61]:
def generate_recommendation(model, user_id, ratings_df, n_items):
    movie_ids = ratings_df["movieId"].unique()
    movie_ids_user = ratings_df.loc[ratings_df["userId"] == user_id, "movieId"]
    movie_ids_to_pred = np.setdiff1d(movie_ids, movie_ids_user)
     
    # Apply a rating of 4 to all interactions (only to match the Surprise dataset format)
    test_set = [[user_id, movie_id, 4] for movie_id in movie_ids_to_pred]
    
    predictions = model.test(test_set)
    
    pred_ratings = np.array([pred.est for pred in predictions])
    index_max = (-pred_ratings).argsort()[:n_items]

    print("Top {0} item recommendations for user {1} ({2}):".format(n_items, get_user_name(user_id), user_id))
    for i in index_max:
       movie_id = movie_ids_to_pred[i]
       print(f"{get_movie_name(movie_id):<50} {round(pred_ratings[i], 3):>6}")
 
 
userID = get_user_id("liannehr")
n_items = 100

generate_recommendation(svd, userID, df, n_items)

Top 100 item recommendations for user liannehr (10989):
babylon-2022                                        4.799
mommy-2014                                          4.643
portrait-of-a-lady-on-fire                          4.572
jojo-rabbit                                         4.493
big-little-lies                                      4.48
american-honey                                      4.473
the-haunting-2018                                   4.466
schindlers-list                                     4.464
city-of-god                                         4.463
chernobyl                                           4.439
little-miss-sunshine                                4.438
the-lord-of-the-rings-the-return-of-the-king        4.434
waves-2019                                          4.427
bo-burnham-inside                                   4.406
miss-americana                                      4.401
the-queens-gambit                                   4.399
12-angry-men    

In [62]:
userID = get_user_id("sverlaan")
n_items = 100

generate_recommendation(svd, userID, df, n_items)

Top 100 item recommendations for user sverlaan (10990):
oj-made-in-america                                  4.688
mommy-2014                                          4.543
normal-people-2020                                  4.457
twin-peaks-the-return                               4.448
amadeus                                             4.432
over-the-garden-wall-2014                           4.418
witness-for-the-prosecution-1957                    4.402
seven-samurai                                       4.388
singin-in-the-rain                                  4.379
sharp-objects                                       4.364
dog-day-afternoon                                    4.36
a-separation                                        4.354
stop-making-sense                                   4.344
big-little-lies                                     4.332
paper-moon                                          4.328
the-wages-of-fear                                   4.325
short-term-12-20

In [76]:
# TODO: maak de similarity hier beneden onderdeel van de specifieke user (bijv. top-3 meest similar met je 5/4.5/4 ster gerated films)

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def get_similar_movies(movie_id, model, trainset, top_n=5):
    movie_id_mapping = {inner_id: trainset.to_raw_iid(inner_id) for inner_id in trainset.all_items()}
    
    if movie_id not in movie_id_mapping.values():
        return "Movie ID not in training set"

    inner_id = trainset.to_inner_iid(movie_id)

    movie_embeddings = model.qi
    
    similarities = cosine_similarity([movie_embeddings[inner_id]], movie_embeddings)[0]
    similar_indices = similarities.argsort()[::-1][1:top_n+1]
    
    similar_movie_ids = [movie_id_mapping[idx] for idx in similar_indices]

    return similar_movie_ids


movie_id = get_movie_id("dead-man")

similar_movies = get_similar_movies(movie_id, svd, trainset, 5)
print(f"Movies similar to {get_movie_name(movie_id)}")
for movie in similar_movies:
    print(get_movie_name(movie))

Movies similar to dead-man
down-by-law
fireworks-1997
the-life-aquatic-with-steve-zissou
ghost-dog-the-way-of-the-samurai
mystery-train


# store and load model

In [70]:
# Import Pickle
import pickle

# Save the model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(svd, file)

In [71]:
# Load the model using Pickle
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [74]:
userID = get_user_id("sverlaan")
n_items = 10

# You need the loaded model and df with ratings to get new recommendations. df should contain the user for which to make a prediction
# Also need the user and movie ID mapping
# all this can be computed locally when personalized recommendations is requested
generate_recommendation(loaded_model, userID, df, n_items)

Top 10 item recommendations for user sverlaan (10990):
oj-made-in-america                                  4.688
mommy-2014                                          4.543
normal-people-2020                                  4.457
twin-peaks-the-return                               4.448
amadeus                                             4.432
over-the-garden-wall-2014                           4.418
witness-for-the-prosecution-1957                    4.402
seven-samurai                                       4.388
singin-in-the-rain                                  4.379
sharp-objects                                       4.364


In [84]:
movie_id = get_movie_id("the-worst-person-in-the-world")

# You need the loaded model, the trainset, and movie ID to and from movie Name mapping dicts
similar_movies = get_similar_movies(movie_id, loaded_model, trainset, 10)
print(f"Movies similar to {get_movie_name(movie_id)}")
for movie in similar_movies:
    print(get_movie_name(movie))

Movies similar to the-worst-person-in-the-world
past-lives
shiva-baby-2020
marriage-story-2019
aftersun
tar-2022
minari
before-sunset
cmon-cmon
lady-bird
before-sunrise


In [16]:
from matrix_factorization import BaselineModel, KernelMF, train_update_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

X = movie_data[["user_id", "item_id"]]
y = movie_data["rating"]


#Prepare data for online learning
(
    X_train_initial,
    y_train_initial,
    X_train_update,
    y_train_update,
    X_test_update,
    y_test_update,
) = train_update_test_split(movie_data, frac_new_users=0.2)

# Prepare data
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

977616      7
739419      4
3154009    10
2983607     8
1987385     8
           ..
2681053     9
2488012     8
102519      6
1098108     5
563446      8
Name: rating, Length: 2620180, dtype: int64

TypeError: fit() takes exactly 2 positional arguments (3 given)

In [17]:
# Initial training
matrix_fact = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact.fit(X_train_initial, y_train_initial)

TypeError: can't unbox array from PyObject into native value.  The object maybe of a different type

In [123]:
baseline_model = BaselineModel(method='sgd', n_epochs = 20, reg = 0.005, lr = 0.01, verbose=1)
baseline_model.fit(X_train.to_numpy(), y_train)

pred = baseline_model.predict(X_test)
rmse = mean_squared_error(y_test, pred, squared = False)

print(f'\nTest RMSE: {rmse:.4f}')

AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [112]:
# Initial training
matrix_fact = KernelMF(n_epochs=20, n_factors=100, verbose=1, lr=0.001, reg=0.005)
matrix_fact.fit(X_train_initial, y_train_initial)


NameError: name 'X_train_initial' is not defined